In [1]:
!pip install deepface opencv-python-headless imgaug rembg tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.7 MB/s eta 0:00:00


In [2]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from deepface import DeepFace
# from imgaug import augmenters as iaa
!pip install albumentations
import albumentations as A
!pip install onnxruntime
from rembg import remove  # background removal

25-11-05 14:22:03 - Directory /root/.deepface has been created
25-11-05 14:22:03 - Directory /root/.deepface/weights has been created
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
video_path = "/content/drive/MyDrive/pratik/videos/bikash.mp4"  # <-- change this
output_dir = "/content/drive/MyDrive/pratik/dataset/bikash"
os.makedirs(output_dir, exist_ok=True)

In [5]:
def resize_uniform(img, size=(224, 224)):
    """
    Resize image to target size while preserving aspect ratio,
    padding with black background if needed.
    """
    h, w = img.shape[:2]
    target_w, target_h = size
    scale = min(target_w / w, target_h / h)
    new_w, new_h = int(w * scale), int(h * scale)

    resized = cv2.resize(img, (new_w, new_h))
    new_img = np.zeros((target_h, target_w, 3), dtype=np.uint8)
    x_offset = (target_w - new_w) // 2
    y_offset = (target_h - new_h) // 2
    new_img[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized
    return new_img

In [6]:
def detect_and_crop_face(frame):
    """
    Detect face, crop it, remove background, return processed face image.
    """
    try:
        detections = DeepFace.extract_faces(img_path=frame, detector_backend='retinaface', enforce_detection=False)
        if detections:
            face = detections[0]["face"]
            face = (face * 255).astype("uint8")

            # Resize to uniform size
            face_resized = resize_uniform(face, (224, 224))
            return face_resized
    except Exception as e:
        print("Detection error:", e)
    return None

In [7]:
augmenter = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.7),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
])

def apply_augmentations(face):
    aug = augmenter(image=face)
    return aug['image']

In [8]:
cap = cv2.VideoCapture(video_path)
frame_rate = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(frame_rate)  # capture 1 frame per second if 30fps

count = 0
saved = 0

print("🚀 Processing video and extracting cleaned faces...")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if count % frame_interval == 0:
        face = detect_and_crop_face(frame)
        if face is not None:
            # Save original clean face
            # cv2.imwrite(f"{output_dir}/face_{saved}.jpg", face)
            # saved += 1

            # # Generate 3 augmentations
            # faces_aug = augmenter(images=[face] * 3)
            # for aug_face in faces_aug:
            #     cv2.imwrite(f"{output_dir}/face_{saved}.jpg", aug_face)
            #     saved += 1
            # Original face
            cv2.imwrite(f"{output_dir}/face_{saved}.jpg", face)
            saved += 1

            # 3 augmented variants
            for i in range(3):
                aug_face = apply_augmentations(face)
                cv2.imwrite(f"{output_dir}/face_{saved}.jpg", aug_face)
                saved += 1


    count += 1

cap.release()
print(f"✅ Done! Saved {saved} , 224x224 faces in {output_dir}")

🚀 Processing video and extracting cleaned faces...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5


25-11-05 14:24:52 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


100%|██████████| 119M/119M [00:00<00:00, 259MB/s]


✅ Done! Saved 64 , 224x224 faces in /content/drive/MyDrive/pratik/dataset/bikash
